In [143]:
import os

import pandas as pd
import json
from tqdm import tqdm
import glob

In [144]:
with open("../config.json","r") as f0:
    config = json.load(f0)

JSON_SAVE_LOC = "json_data"

In [145]:
song_details = pd.read_excel("Metarphors_180118T.xlsx",sheet_name="song details")
metaphor_details = pd.read_excel("Metarphors_180118T.xlsx",sheet_name="metaphors")

In [146]:
song_details.head()

,ID,Song Name,Voice,Music,Lyricist,Album,Year,Lyrics
0,1,මගෙ ජීවිනේ,මංජුල පුෂ්පකුමාර,නුසින්‌ නිරෝෂණ,සජිත්‌ දිල්පහන්‌,Null,2010,මගෙ ජීවිනේ ... සොඳුරේ ඔබ පමණයි\nනෑ යන්නෙ.... ...
1,2,හිතුවෙවත්‌ නැති දවසක\n,"මහේෂ්‌ ජයවර්ධන,ශාලිත ශහෙන්ද්‍ර",WIDE ANGLE PRODUCTIONS,Spin Circuitz,Season 1,2010,හිතුවෙවත් නැති දවසක\nහිතන් නැතිවම ලං වුනා\nඅතී...
2,3,රැස්‌ විහිදෙන සමනලියකගෙන්‌,"භානිය, සන්තුෂ්‌",Null,"භානිය, සන්තුෂ්‌",RESVIHIDENA the Valentine's Day Special,2007,රැස් විහිදෙන සමනලියකගෙන්\nඉගියක් ලැබුනාම\nරැක ...
3,4,හිරු මැක්‌ යයි,අමිල පෙරේරා,මහේෂ්‌ දෙනිපිටිය,ආරියසිංහ පෙරේරා,ආසයි මං පියාඹන්න,2007,හිරු මැකී යයි නීල අහසේ සේද මීදුම්‌ මායිමේ\nනිම...
4,5,චන්ද්රයන් පිදූ,Daddy,Null,Null,Massina,2010,සිහිනෙන් වූවත් නෑ මා දුටුවේ\nමාගේ දිවියේ අන්ධක...


In [147]:
metaphor_details.head()

,Song_ID,Metaphor,Meaning,source domain,target domain
0,1,තරු මිලින වෙලා සඳු නෑ අද මගෙ ලොකෙ,I am sad as I have lost my happiness.,Moon,Lover
1,1,තව එක මොහොතක්‌ නුබෙ සෙනෙහේ මගෙ වේනම්‌ තරු අතූර...,I will do anything for your love becomes mine ...,stars in sky,Effort
2,2,සමනල්ලු ඉහ හැරුණු අත ගියා අපි ආයේ නොයෙනා නිසා,Love is lost as we won't meet again.,Butterfly flight,drifting apart
3,3,රැස් විහිදෙන සමනලියකගෙන් ඉගියක් ලැබුනාම,A beautiful girl gets interested in you,radiant butterfly,Lover
4,3,දිරි දෙන්නම් මුළු හද පත්ලෙන් මහමෙර සේම,Will give you encouragement(like the ocean).,Ocean,Effort


In [148]:
metaphor_details.shape,song_details.shape

((144, 5), (100, 8))

In [149]:
def concat_dict(dict_a:dict,dict_b:dict) -> dict:
    return_dict = {}

    for k in dict_a.keys():
        return_dict[k] = dict_a.get(k) if dict_a.get(k) else "Null"

    for k in dict_b.keys():
        return_dict[k] = dict_b.get(k) if dict_b.get(k) else "Null"

    return return_dict

def data_request_format(configuration:dict,value_dict:dict):
    return {
        "_index":configuration["elasticsearch"]["index"],
        "_source":value_dict
    }


def gen_song_jsons(metaphor_df:pd.DataFrame,song_df:pd.DataFrame,save_fol) -> None:
    for row_ind in tqdm(range(metaphor_df.shape[0]),desc="Processing Songs",total=metaphor_df.shape[0]):
        row = metaphor_df.loc[row_ind,:].to_dict()
        song_d = song_df.loc[row["Song_ID"]-1,:].to_dict()
        combined_dict = concat_dict(song_d,row)
        request_format = data_request_format(config,combined_dict)

        with open(os.path.join(save_fol,f"{row_ind}.json"),"w") as fileO:
            json.dump(request_format,fileO)

In [150]:
gen_song_jsons(metaphor_details,song_details,JSON_SAVE_LOC)

Processing Songs: 100%|██████████| 144/144 [00:00<00:00, 1133.87it/s]


## Send Request to Elastic

In [151]:
list_files = [x for x in os.listdir(JSON_SAVE_LOC) if ".json" in x]

In [152]:
def datagen(file_list:list):
    for f in file_list:
        with open(os.path.join(JSON_SAVE_LOC,f),"r") as fileO:
            data = json.load(fileO)

        yield data

In [153]:
file_ = next(datagen(list_files))

In [154]:
file_

{'_index': 'sinhalasongmetaphorsdb',
 '_source': {'ID': 1,
  'Song Name': 'මගෙ ජීවිනේ',
  'Voice': ' මංජුල පුෂ්පකුමාර',
  'Music': 'නුසින්\u200c නිරෝෂණ',
  'Lyricist': 'සජිත්\u200c දිල්පහන්\u200c',
  'Album': 'Null',
  'Year': 2010,
  'Lyrics': 'මගෙ ජීවිනේ ...  සොඳුරේ ඔබ පමණයි\nනෑ යන්නෙ.... හැරදා... කිසිදා...\n\nතරු මිලින වෙලා සඳු නෑ අද මගෙ ලොකෙ\nබොඳවී පැතූමන්\u200c මා හඩවා...//\n\nගැස්සෙන ඔය හදේ රිද්මයේ\nසත්තට තූරුළු ඵී සැනහුනේ\n\nලස්සන සිනහවක පැටලුනේ\nසිඳගෙන ඔබද මා සුරතලිී...//\n\nනවතා සුසුමන්\u200c හැර යන්න එපා\n\nඔබමයි ප්\u200dරියේ මාගේ ජීවේ\n\nතව එක මොහොතක්\u200c නුබෙ සෙනෙහේ මගෙ වේනම්\u200c\nතරු අතූරා අහසේ සරසන්නම්\u200c...//\n\nගැස්සෙන ඔය හදේ රිද්මයේ\nසත්තට තූරුළු ඵී සැනහුනේ\n\nලස්සන සිනහවක පැටලුනේ\nසිඳගෙන ඔබද මා සුරතලිී...//',
  'Song_ID': 1,
  'Metaphor': 'තරු මිලින වෙලා සඳු නෑ අද මගෙ ලොකෙ',
  'Meaning': 'I am sad as I have lost my happiness.',
  'source domain': 'Moon',
  'target domain': 'Lover'}}

In [155]:
config["elasticsearch"]

{'host': 'https://localhost',
 'port': 9200,
 'username': 'elastic',
 'password': 'R-6ou__S5RFYOospV0vY',
 'index': 'sinhalasongmetaphorsdb',
 'api_key': {'id': 'e-cBwYUBOVUGeTIumKDk',
  'name': 'search-api-key',
  'expiration': 1674585185736,
  'api_key': 'CQT9tphlQTej-53I43F11Q',
  'encoded': 'ZS1jQndZVUJPVlVHZVRJdW1LRGs6Q1FUOXRwaGxRVGVqLTUzSTQzRjExUQ=='}}

In [156]:
from elasticsearch import Elasticsearch,helpers

client = Elasticsearch(HOST=config["elasticsearch"]["host"], PORT=config["elasticsearch"]["port"],http_auth=(config["elasticsearch"]["username"],config["elasticsearch"]["password"]),scheme="https",ca_certs="../elasticsearch-8.6.0-windows-x86_64/elasticsearch-8.6.0/config/certs/http_ca.crt")

INDEX = config["elasticsearch"]["index"]

In [157]:
helpers.bulk(client,datagen(list_files))

(144, [])